In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

In [3]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
tsla.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, TSLA)   1551 non-null   float64
 1   (High, TSLA)    1551 non-null   float64
 2   (Low, TSLA)     1551 non-null   float64
 3   (Open, TSLA)    1551 non-null   float64
 4   (Volume, TSLA)  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [7]:
len(tsla)

1551

In [8]:
xly.head()

Price,Close,High,Low,Open,Volume
Ticker,XLY,XLY,XLY,XLY,XLY
Date,,,,,
2019-01-02,94.046104,94.574027,91.246214,91.745857,6840800
2019-01-03,92.009834,93.555901,91.849573,93.329649,6346000
2019-01-04,95.054802,95.752415,93.065655,93.254197,7269100
2019-01-07,97.204231,97.826425,95.460194,95.507326,6263100
2019-01-08,98.278931,98.806854,96.883697,98.212940,9391000


In [9]:
len(xly)

1551

In [10]:
high = tsla[('High', 'TSLA')]
low = tsla[('Low', 'TSLA')]

In [11]:
def calculate_vortex(df, n=14):
    high = df[("High", "TSLA")]
    low = df[("Low", "TSLA")]
    close = df[("Close", "TSLA")]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [12]:
calculate_vortex(tsla)[0]

Date
2019-01-02         NaN
2019-01-03         NaN
2019-01-04         NaN
2019-01-07         NaN
2019-01-08         NaN
                ...   
2025-02-26    0.620144
2025-02-27    0.651928
2025-02-28    0.634852
2025-03-03    0.703732
2025-03-04    0.732949
Length: 1551, dtype: float64

In [13]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=demo'
r = requests.get(url)
sentiment = r.json()

In [14]:
print(sentiment)

{'items': '50', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': 'New Inflation Data Dismays Bulls, CoreWeave IPO Shows Cooling AI Frenzy, Lululemon Shows Cautious Consumer Spending', 'url': 'https://www.benzinga.com/markets/equities/25/03/44540673/new-inflation-data-dismays-bulls-coreweave-ipo-shows-cooling-ai-frenzy-lululemon-shows-cautious-', 'time_published': '20250328T161558', 'authors': ['The Arora Report'], 'summary': 'To gain an edge, this is what you need to know today. Please click here for an enlarged chart of SPDR S&P 500 ETF Trust ( SPY ) which represents the benchmark stock market index S&P 500 ( SPX ) . Core PCE came in at 0.4% vs 0.4% consensus, but whisper numbers were around 0.3%.', 'banner_image': 'https://thearorareport.com/wp-conte